# Function to fit our analytical model to the SF-gas from TNG

Our fit procedure is also described in section 3 of van son et al in prep.




In [8]:
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from scipy import interpolate
from scipy.optimize import minimize

# from astropy.cosmology import WMAP9, z_at_value
from astropy.cosmology import Planck18  as cosmo# Planck 2018
from astropy.cosmology import z_at_value

############################
# Custom scripts
import paths
import get_ZdepSFRD as Z_SFRD
import importlib
importlib.reload(Z_SFRD)

Cosmol_sim_location = '/Users/lieke/surfdrive/Documents/CompareCOMPAS/' + "SFRMetallicityFromGasTNG100.hdf5"


# First read the TNG data 
## And convert it to a SFDR in Msun/yr/Mpc^-3

***
We will have simulation data == TNG in our case
and model data  == our analytical function
***


In [9]:
##########################################
# Simulated SFRD data (from TNG)
##########################################
## Read in the pure simulation data
with h5.File(Cosmol_sim_location, "r") as f:
    MetalBins     = f["MetalBins"][:]
    Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr        = f["Sfr"][:]
# Convert SFR from sfr/box to sfr Mpc-3
# littleh  = 0.6774
# Rbox     = 75/littleh
# Sim_SFRD = BoxSfr / Rbox**3 *u.Mpc**-3
Sim_SFRD = BoxSfr#Sim_SFRD.value

## Adjust what metallicities to include 
# The minimum metallicity in COMPAS is 10^-4, so there is no
# use in fitting to the 10^-7 metallicity tail
Sim_center_Zbin = (MetalBins[:-1] + MetalBins[1:])/2.
low_bound_Z_ind  = np.where(Sim_center_Zbin > 1e-5)[0]# index of Sim_center_Zbin, where Z > 1e-5
tofit_Sim_metals = Sim_center_Zbin[low_bound_Z_ind]   

## Reverse the time axis of the SFRD and lookback time for the fit
tofit_Sim_SFRD      = Sim_SFRD[:,low_bound_Z_ind][::-1]
tofit_Sim_lookbackt = Lookbacktimes[::-1] 

## Convert lookback times to redshifts
# the last value of Lookbacktimes = 0, which is problematic for z calculation
redshifts_Sim = [z_at_value(cosmo.lookback_time,t*u.Gyr) for t in Lookbacktimes[:-1]] 
redshifts_Sim.append(0) # put redshift zero back at the end
redshifts_Sim = np.array(redshifts_Sim)


# Interpolate the TNG data

Using scipy interpolate
https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html

f = interpolate.interp2d(x, y, z)



In [3]:
#########################################
# Interpolate the simulation data
f_interp = interpolate.interp2d(tofit_Sim_lookbackt, tofit_Sim_metals, tofit_Sim_SFRD.T, kind='cubic')

#########################################
# Retrieve values at higher res regular intervals
redshift_interp_Sim         = np.arange(0, 10.1, 0.05)
Lookbacktimes_interp_Sim    = [cosmo.lookback_time(z).value for z in redshift_interp_Sim]

log_tofit_Sim_metals        = np.log10(tofit_Sim_metals)
metal_interp_Sim            = np.logspace(min(log_tofit_Sim_metals), max(log_tofit_Sim_metals), 100)

SFRD_interp_Sim             = f_interp(Lookbacktimes_interp_Sim,metal_interp_Sim)

print('np.shape(Lookbacktimes_interp_Sim)', np.shape(Lookbacktimes_interp_Sim),'np.shape(metal_interp_Sim)',
      np.shape(metal_interp_Sim), 'np.shape(SFRD_interp_Sim)', np.shape(SFRD_interp_Sim) )


np.shape(Lookbacktimes_interp_Sim) (202,) np.shape(metal_interp_Sim) (50,) np.shape(SFRD_interp_Sim) (50, 202)


In [24]:

##################################################
# muz =-0.09, mu0 =0.026, sigma =1.9, alpha=-3.3
##################################################
def calc_chi_square(simulation_SFRD = [], simulation_metals = [], simulation_redshifts = [],  
                    mu_0_list = 0.026, muz_list =-0.09, sigma0_list = 1.9, sigmaz_list = 1.9, alpha_list =-3.3,
                    sf_a =0.01 , sf_b=2.6, sf_c=3.2 , sf_d=6.2):
    """
    Calculate the distribution of metallicities at different redshifts using a log skew normal distribution
    that is basically a skew normal distribution, but then with the random variable x = ln(Z)

    NOTE: This assumes that metallicities in COMPAS are drawn from a flat in log distribution

    Args:
        fit_metals              --> [float]          metals used for fitt
        Redshifts               --> [float]          redshihts used to fit
        simulation_SFRD           --> [float]        cosmological simulation SFRD to fit to
        
        mu_0_list    = -0.23    --> [float]          location (mean in normal) at redshift 0
        muz_list = 0.035    --> [float]          redshift evolution of the location
        sigma0_list  = 0.39     --> [float]          Scale at redshift 0 (variance in normal)
        sigmaz_list  = 0.0      --> [float]          redshift evolution of Scale (variance in normal)
        alpha_list   = 0.0      --> [float]          shape (skewness, alpha = 0 retrieves normal dist)

        sf_a                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_b                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_c                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_d                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)

    Returns:
        tot_chi_square          --> [float ] 
    """
    
    """
    ######################################

    """  
    #print('muz_list', muz_list, 'mu_0_list',mu_0_list, 'sigma0_list',sigma0_list,'sigmaz_list',sigmaz_list, 'alpha_list',alpha_list,
    #      'sf_a', sf_a, 'sf_b', sf_b, 'sf_c', sf_c, 'sf_d', sf_d)
        
    #####################################
    # Get the SFR
    # Madau & Fragos 2017: a=0.01, b=2.6, c=3.2,  d=6.2
    sfr = Z_SFRD.Madau_Dickinson2014(simulation_redshifts, a=sf_a, b=sf_b, c=sf_c, d=sf_d) # Msun year-1 Mpc-3 

    # Get dPdZ 
    dPdlogZ, redshifts, metallicities, step_logZ, p_draw_metallicity = \
                    Z_SFRD.skew_metallicity_distribution(mu_z = muz_list, mu_0 = mu_0_list,
                                                  sigma_0= sigma0_list, sigma_z=sigmaz_list, alpha = alpha_list, 
                                                  metals=simulation_metals, redsh = simulation_redshifts)
        
    log_fit_metallicities = np.log10(simulation_metals)
    step_fit_logZ         = np.diff(log_fit_metallicities)[0]
    
    ######################################
    # For each redshift in the TNG data:
    tot_chi_square = 0
        
    for redshift_i in range(len(redshifts)):
        #print(redshift_i, 'at redshift', redshifts[redshift_i])
        ######################################
        # Now the SFRD = sfr x dPdZ
        model_SFRD = sfr[redshift_i] *dPdlogZ[redshift_i,:] # Msun year-1 Mpc-3 

        # Model comes in dP/dlogZ, so should your sim-data 
        sim   = simulation_SFRD[redshift_i,:]/step_fit_logZ  # 
        model = model_SFRD.value 
        
        ###################
        # Zero values are troublesome in Chi_squared!
        # only fit where sim > x 
        common_bool = np.logical_and(model > 1e-11, sim > 1e-11)
        model       = model[common_bool]
        sim         = sim[common_bool]

        ###################
        # Actual CHI_squared
        # Squared residuals at this redshift
        res_squared = (sim - model )**2
        
#         print('max(res_squared)', max(res_squared))
        # Sum of squared residuals
        tot_chi_square += np.sum(res_squared, axis = -1)/np.sum(model) 


    ######################################
    # Minimum Chi_squared taking all redshift into aaccount
    return tot_chi_square


    
    

# Run your chi square calculations

## and leave the refinement up to scipy minimize

In [25]:
#################################################################
## Function wrapper to minimize the Chi_square
#################################################################
def test_chi(x0 = [-0.09, 0.026, 1.9, 0.1, -3.3,0.01, 2.6, 3.2, 6.2] ):
    chi_square_matix = calc_chi_square(simulation_SFRD = SFRD_interp_Sim.T, simulation_metals = metal_interp_Sim, simulation_redshifts = redshift_interp_Sim, 
                                       muz_list =x0[0], mu_0_list =x0[1],sigma0_list =x0[2], sigmaz_list=x0[3], alpha_list =x0[4],
                                       sf_a =x0[5], sf_b=x0[6], sf_c=x0[7], sf_d=x0[8])
    
    return chi_square_matix


# BEST GUESS
x0 = np.array([-0.15, 0.026, 1.1, 0.1, -3.3, 0.01, 2.6, 3.2, 6.2])
# FIT
res = minimize(test_chi, x0= x0, method ='nelder-mead', options = {'maxiter': 5000})




/Users/lieke/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
print(res.success, res.message, res.nit)
muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best = res.x[0], res.x[1], res.x[2], res.x[3],res.x[4]
sf_a_best, sf_b_best, sf_c_best, sf_d_best    = res.x[5], res.x[6], res.x[7], res.x[8] 

print('\nBEST FITTING PARAMETERS:')
print('muz =%s, mu0 =%s, sigma_0 =%s, sigma_z =%s, alpha=%s'% (muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best) )
print('sf_a =%s, sf_b =%s, sf_c =%s, sf_d =%s'% (sf_a_best, sf_b_best, sf_c_best, sf_d_best) )



True Optimization terminated successfully. 1717

BEST FITTING PARAMETERS:
muz =-0.04677378362710756, mu0 =0.024730821484976583, sigma_0 =1.1166378490971502, sigma_z =0.048190741547454244, alpha=-1.7086079887680787
sf_a =0.06570561539662137, sf_b =1.4831498367401643, sf_c =4.44988154529773, sf_d =5.905882609485851


In [ ]:
if res.success:
    np.savetxt(paths.data / 'best_fit_parameters.txt', (mu0_best, muz_best, sigma0_best, sigmaz_best, alpha_best,sf_a_best, sf_b_best, sf_c_best, sf_d_best),fmt="%s")
